# Assignment - to build Golden Data Set using Synthetic Data Generation with RAGAS  and using different retrievers and with RAG Evaluation with LangSmith

The following retrievers are evaluated with Semantic Chunking enabled / disabled options.

## Install dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### Import NLTK

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DadaV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DadaV\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Load LangChain API key

In [29]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

Create a unique project name

In [30]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]
project_name = os.environ["LANGCHAIN_PROJECT"] = f"Philippines AI Bills RAG - {unique_id}"
project_name

'Philippines AI Bills RAG - d3128391'

Loan OpenAI and Cohere API Keys

In [31]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [32]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [33]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader

path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [34]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
# generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [35]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [36]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [37]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node fe592888-425b-4e5a-926e-602fd892e08e does not have a summary. Skipping filtering.
Node 59e4e8ec-db7c-4091-b6c3-7623f2054cab does not have a summary. Skipping filtering.
Node b6bc7938-2a3d-4b9e-8c10-882344b0eb1f does not have a summary. Skipping filtering.
Node 9e086dc2-b42b-47d4-abf0-c12b813309c1 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 138)

We can save and load our knowledge graphs as follows.

In [38]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 138)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [39]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [40]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

Finally, we can use our `TestSetGenerator` to generate our testset!

In [ ]:
# testset = generator.generate(testset_size=20, query_distribution=query_distribution)
# testset.to_pandas()
# testset.to_jsonl("bills/golden_dataset.json")  # Save for reuse

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

KeyError: "No persona found with name 'AI Policy Stakeholder'"

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [41]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 617ef8b9-6c3a-4626-a5ca-36de91c126a6 does not have a summary. Skipping filtering.
Node 7ee28968-dc61-4ff6-9bf5-554b26c21a7d does not have a summary. Skipping filtering.
Node 6217a586-49f8-433d-9904-c01ca4fbdb41 does not have a summary. Skipping filtering.
Node 641bdc81-ff9d-47ec-a29a-d9b9edb3eb6f does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [42]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of CD0 Magazine in discussing...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,"According to the provided context, CD0 Magazin...",single_hop_specifc_query_synthesizer
1,How does the bill aim to balance AI developmen...,[AI presents enormous opportunities for the Ph...,The bill seeks to strike a careful balance bet...,single_hop_specifc_query_synthesizer
2,What is the significance of the Republic of th...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The context mentions the Republic of the Phili...,single_hop_specifc_query_synthesizer
3,How does the legislatoin aim to regulate AI in...,"[1 \na) Promote innovation, technological adva...",The legislation aims to regulate the developme...,single_hop_specifc_query_synthesizer
4,How do the legal provishuns and enforcement of...,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The legal provisions and enforcement measures ...,multi_hop_abstract_query_synthesizer
5,How does the need for regulation and oversight...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The context shows that regulation and oversigh...,multi_hop_abstract_query_synthesizer
6,How do reguLation of AI use by empLoyers and b...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The regulation of AI use by employers and busi...,multi_hop_abstract_query_synthesizer
7,How does the AI Regulation Act (AIRA) promote ...,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The AI Regulation Act (AIRA) promotes AI in pu...,multi_hop_abstract_query_synthesizer
8,How does the NAIC's regulation of AI in employ...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The NAIC's regulation of AI in employment and ...,multi_hop_specific_query_synthesizer
9,How do Sections 8 and 17 of the AI Act collect...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,Section 8 of the AI Act details the powers and...,multi_hop_specific_query_synthesizer


## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [43]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills " + unique_id

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

dataset_name

'Philippines AI Bills d3128391'

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [44]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [45]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

p_chunk_size = 1000 # 500 in previous try

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = p_chunk_size,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [47]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

For the prompt, we will make it switchable to use with empathy version or non-empathy standard version. 

In [54]:
from langchain.prompts import ChatPromptTemplate

empathy = True

if empathy:

    RAG_PROMPT = """\
    Given a provided context and question, you must answer the question based only on context.

    If you cannot answer the question based on the context - you must say "I don't know".

    You must answer the question using empathy and kindness, and make sure the user feels heard.

    Context: {context}
    Question: {question}
    """

    # rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

else:

    RAG_PROMPT = """\
    You are a helpful and informative assistant. Given a provided context and question, you must answer the question based only on context.

    If you cannot answer the question based on the context, or you are unsure, you must say "I don't know".

    Context: {context}
    Question: {question}
    """

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)



For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [49]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4.1-mini")
chat_model = ChatOpenAI(model="gpt-4.1-nano")

As usual, we will power our RAG application with Qdrant!

Here we have version with semantic chunking and another version with recursive text splitter

In [50]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

semantic_documents = semantic_chunker.split_documents(rag_documents[:20])

# semantic chunking OFF - use rag_documents, else semantic_documents

from langchain_community.vectorstores import Qdrant

text_vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG")

semantic_vectorstore = Qdrant.from_documents(
    documents=semantic_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG")

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [55]:
eval_llm = ChatOpenAI(model="gpt-4.1")

from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator(
    "qa", 
    config={
        "llm" : eval_llm
        },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],      # student's answer (model output)
        "reference": example.outputs["answer"],   # gold / true answer
        "input": example.inputs["question"],         # the question
        }
    )

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
        },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
        }
    )

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
        },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["response"],
        # "reference": example.outputs["answer"], # to silence the warning as it is not normal for empathy to have reference
        "input": example.inputs["question"],
        }    
    )

In [58]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

from langchain.retrievers.multi_query import MultiQueryRetriever

from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from langchain_qdrant import QdrantVectorStore

from langchain.retrievers import EnsembleRetriever

# Test for a single retriever only
# retriever_names = [
#     "Naive"
# ]

# Enable for multi-retriever tests
retriever_names = [
    "Naive", "BM25", "Multi-Query", "Parent-Document", "Contextual Compression", "Ensemble"
]

results = []

import time

for sc in [False, True]:  # semantic chunking off/on
    for retriever_name in retriever_names: # loop through each retriever
    
        start_time = time.time()
        if sc:
            vectorstore = semantic_vectorstore
        else:
            vectorstore = text_vectorstore

        match retriever_name:
        
            case 'Naive':
                retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
                naive_retriever = retriever
            
            case 'BM25':
                retriever = BM25Retriever.from_documents(rag_documents) 
                bm25_retriever = retriever
            
            case 'Contextual Compression':
                compressor = CohereRerank(model="rerank-v3.5")
                retriever = ContextualCompressionRetriever(
                            base_compressor=compressor, base_retriever=naive_retriever)
                compression_retriever = retriever

            case 'Multi-Query':
                retriever = MultiQueryRetriever.from_llm(retriever=naive_retriever, llm=chat_model)
                multi_query_retriever = retriever
                
            case 'Parent-Document':
                parent_docs = rag_documents
                child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

                client = QdrantClient(location=":memory:")

                client.create_collection(
                    collection_name="full_documents",
                    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
                )

                parent_document_vectorstore = QdrantVectorStore(
                    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
                )
                store = InMemoryStore()
                retriever = ParentDocumentRetriever(
                                vectorstore = parent_document_vectorstore,
                                docstore=store,
                                child_splitter=child_splitter)
                retriever.add_documents(parent_docs, ids=None)
                parent_document_retriever = retriever

            case 'Ensemble':
                retriever_list = [naive_retriever, bm25_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
                equal_weighting = [1/len(retriever_list)] * len(retriever_list)
                retriever = EnsembleRetriever(retrievers=retriever_list, weights=equal_weighting)
            
            case ' ':
                retriever = "Invalid option"
        
        from operator import itemgetter
        from langchain_core.runnables import RunnablePassthrough, RunnableParallel
        from langchain.schema import StrOutputParser

        rag_chain = (
            {"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
            | RunnablePassthrough.assign(context=itemgetter("context"))
            | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
        )

        feedback = rag_chain.invoke({"question" : "How does the Philippines regulate the development of AI in the country?"})
        feedback = feedback["response"].content

        # empathy version
        # rag_chain = (
        #         {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        #         | rag_prompt | chat_model | StrOutputParser()
        #     )

        # feedback = rag_chain.invoke({"question" : "How does the Philippines regulate the development of AI in the country?"})
        
        print('Retriever type:', retriever_name, ' || Semantic chunking :', sc, '|| Empathy prompt used :', empathy)
        print('Question : How does the Philippines regulate the development of AI in the country?')
        print('Answer :', feedback)

        latency = time.time() - start_time
        
        evaluate(
            rag_chain.invoke,
            data=dataset_name,
            evaluators=[
                qa_evaluator,
                labeled_helpfulness_evaluator,
                empathy_evaluator],
            metadata={
                "retriever": retriever_name,
                "semantic chunking": sc,
                "empathy_prompt": empathy,
                "answer": feedback
                # "latency": latency
                }
            )

Retriever type: Naive  || Semantic chunking : False || Empathy prompt used : True
Question : How does the Philippines regulate the development of AI in the country?
Answer : Based on the information provided, the Philippines regulates the development of AI through comprehensive legislation that aims to promote ethical, responsible, and innovative AI practices. The law establishes a framework that includes creating an AI Ethics Review Board to set guidelines on safety, ethics, and accountability. It also involves implementing a risk-based regulatory system that classifies AI systems based on their impact, and requires the registration of all AI systems with the National AI Council (NAIC). This registry ensures oversight and compliance before AI systems can be imported, sold, or deployed. The regulation also emphasizes continuous development, including regular reviews of the AI Roadmap, and encourages supporting Filipino AI innovations through partnerships and grants. Overall, the regula

0it [00:00, ?it/s]

Retriever type: BM25  || Semantic chunking : False || Empathy prompt used : True
Question : How does the Philippines regulate the development of AI in the country?
Answer : Based on the provided context, the Philippines regulates the development of AI by implementing a comprehensive act that covers all types of AI, including advanced forms like AGI and ASI, as well as foundational models such as machine learning systems, neural networks, and generative AI. The regulation applies to all individuals, corporations, institutions, and government agencies involved in developing, deploying, or operating AI systems within the country. The focus is on ensuring that AI development aligns with principles of transparency, accountability, and sustainability, and that policies are driven by futures thinking, strategic foresight, and proactive risk management. If you need more specific details, I'm here to help!
View the evaluation results for experiment: 'advanced-health-40' at:
https://smith.langch

0it [00:00, ?it/s]

Retriever type: Multi-Query  || Semantic chunking : False || Empathy prompt used : True
Question : How does the Philippines regulate the development of AI in the country?
Answer : Based on the information provided, the Philippines regulates the development of AI through a comprehensive framework outlined in legislation such as the AI Regulation Act and related bills. These laws aim to ensure that AI development and use are conducted responsibly, ethically, and in alignment with national interests. The regulation includes establishing a National AI Registry where all AI systems developed or deployed in the country must be registered, ensuring full compliance before importation, sale, or deployment. Additionally, the legislation emphasizes building institutional and regulatory capacities, fostering transparency, accountability, and sustainability in AI policies. It also promotes the creation of an AI Ethics Review Board to issue guidelines on safety, ethics, and social impact, and incorp

...

0it [00:00, ?it/s]

Retriever type: Contextual Compression  || Semantic chunking : True || Empathy prompt used : True
Question : How does the Philippines regulate the development of AI in the country?
Answer : Based on the information provided, the Philippines regulates the development of AI through the Artificial Intelligence Regulation Act (AIRA). This Act aims to govern the development and use of all types of AI systems in the country, ensuring they are used ethically and responsibly. It covers various AI technologies, including foundational models like machine learning systems, neural networks, and language models. The regulation applies to individuals, corporations, institutions, and government agencies involved with AI in the Philippines. The goal is to promote safe, ethical innovation while aligning with national policies on sustainability and responsible development. If you'd like more details or have specific questions, I'm here to help!
View the evaluation results for experiment: 'perfect-power-

0it [00:00, ?it/s]

Retriever type: Ensemble  || Semantic chunking : True || Empathy prompt used : True
Question : How does the Philippines regulate the development of AI in the country?
Answer : Based on the information provided, the Philippines plans to regulate the development of AI through a comprehensive legal framework called the 'Artificial Intelligence Regulation Act (AIRA).' This legislation aims to ensure that AI development and use are ethical, responsible, and aligned with national interests. It covers all types of AI, including advanced forms like AGI and ASI, as well as foundation models such as machine learning systems and neural networks.

The regulation promotes innovation while emphasizing safety, transparency, accountability, and human oversight. It also establishes institutions like the National AI Commission to coordinate efforts, develop a national AI roadmap, and maintain a registry of AI systems. The act encourages adaptation to evolving AI technologies, strategic foresight in poli

0it [00:00, ?it/s]

****** EVALUATION ENDS HERE ********
